## Question 1:

    **Implement BLEU Score metric. Pre-process the text by lower-casing the text and removing punctuation.**


In [6]:
from collections import Counter
import numpy as np

class BleuTextEvaluator:
    def preprocess_text(self, text):
        """
        Preprocesses text by lowercasing and removing punctuation.
        Args:
            text: The text to preprocess.
        Returns:
            The preprocessed text.
        """
        # Lowercase the text
        text = text.lower()
        # Remove punctuation
        text = ''.join([char for char in text if char.isalnum() or char == ' '])
        return text

    def calculate_modified_precision(self, reference_sentence, generated_sentence, n):
        """
        Calculate modified precision of n-grams between a reference sentence
        and a generated sentence.

        Args:
        reference_sentence (str): The reference sentence.
        generated_sentence (str): The generated sentence to be evaluated.
        n (int): The size of the n-grams.

        Returns:
        float: Modified precision score.
        """

        # Tokenize reference and generated sentences
        reference_tokens = self.preprocess_text(reference_sentence).split()
        generated_tokens = self.preprocess_text(generated_sentence).split()

        # Create n-grams for reference and generated sentences
        reference_ngrams = [tuple(reference_tokens[i:i+n]) for i in range(len(reference_tokens)-n+1)]
        generated_ngrams = [tuple(generated_tokens[i:i+n]) for i in range(len(generated_tokens)-n+1)]

        # Count n-grams occurrences
        reference_counts = Counter(reference_ngrams)
        generated_counts = Counter(generated_ngrams)

        # Calculate clipped counts and total counts
        clipped_counts = sum(min(generated_counts[ngram], reference_counts[ngram]) for ngram in generated_counts)
        total_counts = sum(generated_counts.values())

        # Handle division by zero
        if total_counts == 0:
            return 0.0

        # Calculate modified precision
        precision = clipped_counts / total_counts

        return precision

    def calculate_bleu_score(self, references, candidates, N):
        """
        Calculate BLEU score for a list of reference sentences and a list of candidate sentences.

        Args:
        references (list of str): List of reference sentences.
        candidates (list of str): List of candidate sentences to be evaluated.
        N (int): The size of the n-grams.

        Returns:
        float: BLEU score.
        """
        brevity_penalty = 1
        bleu_scores = []
        for ref, candidate in zip(references, candidates):
            weighted_precision_scores = sum((1/N) * np.log(self.calculate_modified_precision(ref, candidate, int(n_gram)) + \
                                                           10**-6 # Added small noise to avoid numerical instability
                                                          ) for n_gram in range(1,N+1))
            bleu_scores.append(brevity_penalty * np.exp(weighted_precision_scores))
        return bleu_scores

## Question 2

        **Use this implementation to find BLEU Score when x="The boys were playing happily on theground." and y="The boys were playing football on the field.".**

In [11]:
text_evaluator = BleuTextEvaluator()
references = [
    "the boys were playing football on the field."
]
candidates = [
    "the boys were playing happily on the ground."
]
# Size of N-gram
N = 4
bleu_scores = text_evaluator.calculate_bleu_score(references, candidates, N)
print("BLEU Scores: {:.5f}".format(bleu_scores[0]))

BLEU Scores: 0.41113


## Question 3
    **Can you explain why we are taking minimum in numerator in equation 1?**

## Question 4
    **Use your implementation to find BLEU Score between any 5 sentence pairs and explain what are potential disadvantages of using the BLEU Score.**

In [17]:
text_evaluator = BleuTextEvaluator()
references = [
    "the cat sat on the mat",
    "the dog barked loudly",
    "the moon shining is",
    "birds are chirping in the trees",
    "a delicious meal is being prepared"
]

candidates = [
    "a cat sat on the mat",
    "the dog barked loudly",
    "the moon is shining",
    "birds chirp in the trees",
    "a tasty meal is being cooked"
]
N = 4
bleu_scores = text_evaluator.calculate_bleu_score(references, candidates, N)
for score in bleu_scores:
    print("BLEU Score {:.4f}".format(score))

BLEU Score 0.7598
BLEU Score 1.0000
BLEU Score 0.0008
BLEU Score 0.0191
BLEU Score 0.0161


- BLEU relies solely on n-gram precision, which may not capture the semantic similarity between sentences accurately. It doesn't consider word order or sentence structure, leading to potentially misleading scores.
- BLEU tends to favor shorter translations due to the brevity penalty. Longer reference sentences may penalize translations unfairly, especially if they contain additional information not present in the translation